In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

## Functions

In [2]:
def build_schedule(solved_per_problem, evals_per_problem, max_iters = 100, log = True):
    assert(len(solved_per_problem) == len(evals_per_problem))

    prior_likelihood = np.ones(len(solved_per_problem)) / len(solved_per_problem)
    expected_evals = np.zeros(len(solved_per_problem))
    
    schedule = []

    for _ in range(0, max_iters):
        # solve_per_eval = (solved_per_problem.mul(prior_likelihood, axis=0).mean(axis=0) / evals_per_problem.mul(prior_likelihood, axis=0).mean(axis=0))
        solve_per_eval = (solved_per_problem / evals_per_problem).mul(prior_likelihood, axis=0).mean(axis=0)
        idxmax = solve_per_eval.idxmax()

        schedule.append(idxmax)
        expected_evals += evals_per_problem[idxmax].mul(prior_likelihood, axis=0).values

        prior_likelihood = (1 - solved_per_problem[idxmax]).values * prior_likelihood
        prior_likelihood = prior_likelihood / sum(prior_likelihood)

        if log: print(f"choose: {idxmax}, unsolved: {prior_likelihood.sum():.2f}, expected_evals: {expected_evals.mean():.2f}")
    
    return (schedule, prior_likelihood, expected_evals)

In [3]:
# solved_per_problem = pd.DataFrame([[0.5, 0.1], [0.1, 0.5]], index = ["A", "B"]).transpose()
# evals_per_problem = pd.DataFrame([[100, 10], [10, 100]], index = ["A", "B"]).transpose()

# build_schedule(solved_per_problem, evals_per_problem, max_iters=10, log = True)

In [4]:
def evaluate_schedule(solved_per_problem, evals_per_problem, schedule, penalty):
    assert(len(solved_per_problem) == len(evals_per_problem))

    prior_likelihood = np.ones(len(solved_per_problem))
    expected_evals = np.zeros(len(solved_per_problem))

    # setup evaluations per algorithm
    evals_per_algo = {}
    for algo in schedule:
        if algo not in evals_per_algo:
            evals_per_algo[algo] = 0
    
    # setup expected evaluations per restart
    evals_per_restart = []

    for algo in schedule:
        evals_this_run = evals_per_problem[algo].mul(prior_likelihood, axis=0).values
        evals_per_algo[algo] += evals_this_run.mean()
        evals_per_restart.append(evals_this_run.mean())
        
        expected_evals += evals_this_run
        prior_likelihood = (1 - solved_per_problem[algo]).values * prior_likelihood

    ert = pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]

    return (ert, evals_per_algo, evals_per_restart)

## Data

In [100]:
dim = 3
files = Path(f"output-500/{dim}D").glob("*/*.csv")

dfs = []

for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

In [101]:
all_df = pd.concat(dfs, ignore_index=True)

# Just CMA-ES
# all_df = all_df[all_df["param_id"].isin(["default", "2L", "4L", "8L", "16L", "32L", "64L"])]

all_df.head()

,log10_target,solved,evaluations,fid,iid,rep,dim,param_id
0,2.0,True,26,3,101,1,3,Powell
1,1.8,True,26,3,101,1,3,Powell
2,1.6,True,27,3,101,1,3,Powell
3,1.4,True,27,3,101,1,3,Powell
4,1.2,True,27,3,101,1,3,Powell


In [102]:
all_df.iid.max()

600

In [103]:
mean_evals = all_df.groupby(["log10_target", "fid", "param_id"], as_index = False).agg({ # , "iid"
    "evaluations": "mean", 
    "solved": "mean",
})

In [104]:
# all_df.groupby(["log10_target", "fid", "param_id", "solved"], as_index = False).agg({ # , "iid"
#     "evaluations": "mean",
# }).query("log10_target == -8.0 and param_id == 'Powell'").sort_values(["fid", "param_id", "solved"])

In [105]:
# mean_evals["spe"] = mean_evals["solved"] / mean_evals["evaluations"]
mean_evals[np.logical_and(mean_evals["fid"] == 24, mean_evals["log10_target"] == -8.0)]

,log10_target,fid,param_id,evaluations,solved
230,-8.0,24,16L,12024.454,0.006
231,-8.0,24,2L,2560.180,0.000
232,-8.0,24,32L,31705.488,0.002
233,-8.0,24,4L,4132.540,0.002
234,-8.0,24,64L,76172.558,0.002
235,-8.0,24,8L,6438.656,0.000
236,-8.0,24,L-BFGS-B,74.880,0.000
237,-8.0,24,Powell,254.954,0.000
238,-8.0,24,SLSQP,58.810,0.000
239,-8.0,24,default,1684.808,0.002


In [106]:
# mean_evals["problem_id"] = mean_evals.apply(lambda x: f"f{x.fid:.0f}-i{x.iid:.0f}-d5-t{x.log10_target}", axis=1)
mean_evals["problem_id"] = mean_evals.apply(lambda x: f"f{x.fid:02}-d{dim:02}-t{x.log10_target:.1f}", axis=1)
mean_evals.head()

,log10_target,fid,param_id,evaluations,solved,problem_id
0,-8.0,1,16L,1811.234,1.0,f01-d03-t-8.0
1,-8.0,1,2L,607.574,1.0,f01-d03-t-8.0
2,-8.0,1,32L,3464.582,1.0,f01-d03-t-8.0
3,-8.0,1,4L,938.244,1.0,f01-d03-t-8.0
4,-8.0,1,64L,6666.148,1.0,f01-d03-t-8.0


In [107]:
mean_evals = mean_evals.drop(columns = ["log10_target", "fid"]) #, "iid"
mean_evals.head()

,param_id,evaluations,solved,problem_id
0,16L,1811.234,1.0,f01-d03-t-8.0
1,2L,607.574,1.0,f01-d03-t-8.0
2,32L,3464.582,1.0,f01-d03-t-8.0
3,4L,938.244,1.0,f01-d03-t-8.0
4,64L,6666.148,1.0,f01-d03-t-8.0


In [108]:
evals_per_problem = mean_evals.pivot(index = "problem_id", columns = "param_id", values = "evaluations")
evals_per_problem.head()

param_id,16L,2L,32L,4L,64L,8L,L-BFGS-B,Powell,SLSQP,default
problem_id,,,,,,,,,,
f01-d03-t-0.2,257.534,91.060,438.296,130.840,710.290,176.168,8.800,19.084,5.99,58.864
f01-d03-t-0.4,307.024,103.758,528.980,153.070,893.686,210.454,8.872,19.718,6.00,69.164
f01-d03-t-0.6,351.926,118.392,632.806,177.706,1100.666,246.172,8.904,20.094,6.00,79.526
f01-d03-t-0.8,393.934,131.050,718.894,197.542,1270.384,276.732,8.920,20.484,6.00,89.248
f01-d03-t-1.0,437.214,145.308,792.826,218.964,1456.366,308.530,8.944,20.796,6.00,99.438


In [109]:
solved_per_problem = mean_evals.pivot(index = "problem_id", columns = "param_id", values = "solved")
solved_per_problem.head()

param_id,16L,2L,32L,4L,64L,8L,L-BFGS-B,Powell,SLSQP,default
problem_id,,,,,,,,,,
f01-d03-t-0.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d03-t-0.4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d03-t-0.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d03-t-0.8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f01-d03-t-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [110]:
print(solved_per_problem.loc[solved_per_problem.max(1) == 0].index.to_list())

[]


In [111]:
# evals_per_problem.to_csv(Path(f"schedule-data/evals_per_problem-{dim}D.csv"))
# solved_per_problem.to_csv(Path(f"schedule-data/solved_per_problem-{dim}D.csv"))

In [112]:
schedule, post_likelihood, expected_evals = build_schedule(solved_per_problem, evals_per_problem, max_iters = 1000)

choose: SLSQP, unsolved: 1.00, expected_evals: 0.06
choose: 2L, unsolved: 1.00, expected_evals: 1.30
choose: SLSQP, unsolved: 1.00, expected_evals: 1.36
choose: SLSQP, unsolved: 1.00, expected_evals: 1.42
choose: SLSQP, unsolved: 1.00, expected_evals: 1.47
choose: Powell, unsolved: 1.00, expected_evals: 1.62
choose: SLSQP, unsolved: 1.00, expected_evals: 1.68
choose: 4L, unsolved: 1.00, expected_evals: 4.02
choose: Powell, unsolved: 1.00, expected_evals: 4.17
choose: Powell, unsolved: 1.00, expected_evals: 4.32
choose: Powell, unsolved: 1.00, expected_evals: 4.47
choose: Powell, unsolved: 1.00, expected_evals: 4.62
choose: Powell, unsolved: 1.00, expected_evals: 4.77
choose: Powell, unsolved: 1.00, expected_evals: 4.92
choose: Powell, unsolved: 1.00, expected_evals: 5.07
choose: Powell, unsolved: 1.00, expected_evals: 5.23
choose: Powell, unsolved: 1.00, expected_evals: 5.38
choose: Powell, unsolved: 1.00, expected_evals: 5.53
choose: Powell, unsolved: 1.00, expected_evals: 5.69
choose

In [113]:
f"{schedule}"

"['SLSQP', '2L', 'SLSQP', 'SLSQP', 'SLSQP', 'Powell', 'SLSQP', '4L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '8L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', '16L', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell', 'Powell',

In [97]:
pd.value_counts(schedule)

/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_32022/2913986251.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(schedule)
/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_32022/2913986251.py:1: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.value_counts(schedule)


Powell      608
16L         341
2L           17
SLSQP        12
L-BFGS-B     12
32L           7
default       1
4L            1
8L            1
Name: count, dtype: int64

In [98]:
penalty = 10_000_000 * dim

(schedule_ert, epa, epr) = evaluate_schedule(solved_per_problem, evals_per_problem, schedule, penalty)
f"{schedule_ert:_.2f}"

/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_32022/461508285.py:24: RuntimeWarning: divide by zero encountered in divide
  ert = pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]


'1_501_217.68'

In [99]:
expected_evals_schedule = pd.DataFrame({"algorithm": schedule, "expected_evals": epr})
expected_evals_schedule["dim"] = dim
expected_evals_schedule.to_csv(f"expected_evals_schedule_{dim}D.csv", index=False) # .groupby("algorithm").sum("expected_evals").sort_values("expected_evals", ascending=False)

In [46]:
# sbs
all_ert = (evals_per_problem / solved_per_problem).clip(upper=penalty).mean()
all_ert.sort_values()

param_id
default     4.237613e+04
4L          7.330893e+04
16L         1.096272e+05
8L          8.705369e+05
2L          8.936764e+05
32L         1.359184e+06
64L         2.583411e+06
Powell      6.820269e+06
L-BFGS-B    8.190817e+06
SLSQP       9.683095e+06
dtype: float64

In [47]:
sbs = min(all_ert)
f"{sbs:_.2f}"

'42_376.13'

In [48]:
# vbs
vbs = (evals_per_problem / solved_per_problem).clip(upper=penalty).apply(min, axis = 1).mean()
f"{vbs:_.2f}"

'27_339.05'

In [49]:
print(f"SBS-VBS Gap closed (Resubstitution): {100 - 100 * (schedule_ert - vbs) / (sbs - vbs):.2f}%")

SBS-VBS Gap closed (Resubstitution): 59.69%


In [503]:
penalty = 1_000_000 * dim

erts = []

# method = "sbs"
method = "normal"

for i in range(1, 25):
    print(f"Evaluating f{i:02}", end = "\r")
    filter_string = f"f{i:02}-"

    just_fi = [x for x in solved_per_problem.index if x.startswith(filter_string)]
    all_but_fi = [x for x in solved_per_problem.index if not x.startswith(filter_string)]

    if method == "sbs":
        all_ert = (evals_per_problem.loc[all_but_fi] / solved_per_problem.loc[all_but_fi]).clip(upper=penalty).mean()
        schedule = [all_ert.sort_values().index[0]]
        print(schedule)
    else:
        schedule, post_likelihood, expected_evals = build_schedule(solved_per_problem.loc[all_but_fi], evals_per_problem.loc[all_but_fi], max_iters = 1000, log = False)
    
    (ert, epa, epr) = evaluate_schedule(solved_per_problem.loc[just_fi], evals_per_problem.loc[just_fi], schedule, penalty)
    erts.append(ert)

lopo_ert = np.mean(erts)
f"{lopo_ert:_.2f}"

/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_88438/461508285.py:24: RuntimeWarning: divide by zero encountered in divide
  ert = pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]


/var/folders/rc/yjrzy52j3w10rmvsf8ky6qkh0000gn/T/ipykernel_88438/461508285.py:24: RuntimeWarning: divide by zero encountered in divide
  ert = pd.DataFrame((expected_evals / (1 - prior_likelihood))).clip(upper = penalty).mean()[0]


'1_357_066.06'

In [504]:
schedule

['SLSQP',
 'default',
 'SLSQP',
 'SLSQP',
 '4L',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'SLSQP',
 'L-BFGS-B',
 'L-BFGS-B',
 '8L',
 'SLSQP',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 '32L',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',
 'L-BFGS-B',


In [505]:
# erts_sbs = erts

In [506]:
((pd.DataFrame(erts) - pd.DataFrame(erts_sbs)) / pd.DataFrame(erts_sbs)) # .sum()

,0
0,-0.998328
1,-0.903109
2,2.542768
3,0.000067
4,-0.958610
5,-0.882955
6,-0.435450
7,-0.915283
8,-0.954373
9,-0.872493


In [496]:
np.log10(erts).mean()

4.3864335884590915

In [497]:
print(f"SBS-VBS Gap closed (LOPO): {100 - 100 * (lopo_ert - vbs) / (sbs - vbs):.2f}%")

SBS-VBS Gap closed (LOPO): -45.66%


In [360]:
pd.DataFrame(schedule).value_counts()

0     
2L        912
Powell     85
SLSQP       2
4L          1
Name: count, dtype: int64

In [142]:
(solved_per_problem["Powell"] > 0).mean()

0.5555555555555556

In [98]:
((solved_per_problem / evals_per_problem)["32L"] > 0).sort_values()

problem_id
f4-d10-t-8.0     False
f24-d10-t-6.0    False
f24-d10-t-6.2    False
f24-d10-t-6.4    False
f24-d10-t-6.6    False
                 ...  
f16-d10-t1.6      True
f16-d10-t1.4      True
f16-d10-t1.2      True
f17-d10-t-0.6     True
f9-d10-t2.0       True
Name: 32L, Length: 1224, dtype: bool

In [99]:
(solved_per_problem / evals_per_problem)["32L"]

problem_id
f1-d10-t-0.2    0.000281
f1-d10-t-0.4    0.000260
f1-d10-t-0.6    0.000241
f1-d10-t-0.8    0.000226
f1-d10-t-1.0    0.000212
                  ...   
f9-d10-t1.2     0.000181
f9-d10-t1.4     0.000208
f9-d10-t1.6     0.000234
f9-d10-t1.8     0.000260
f9-d10-t2.0     0.000292
Name: 32L, Length: 1224, dtype: float64